In [1]:
import cv2
import torch
import tensorflow as tf
import time
import numpy as np
import pandas as pd
import os
from utility.yolov5_output_processing import processing_yolov5

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

#os.environ['CUDA_VISIBLE_DEVICES'] = "0"

/home/chakradhar/.conda/envs/multimodular/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from body_pose_recognition.body_features_extraction_train_with_images import body_features_extraction

In [3]:
model_yolov5 = torch.hub.load('ultralytics/yolov5', 'yolov5m')

Using cache found in /home/chakradhar/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-6-15 Python-3.9.12 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24260MiB)

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


# Extracting Features and saving to a File

In [4]:
class_index = {'pos' : 1, 'neg': 0}

input_base_path = r'../train_latest'
output_base_path = r'../train csv openpose'

#
# bf for body features
#
bf_ex, num_bf = body_features_extraction()

try:
    os.mkdir(output_base_path)
except Exception as e:
    print("Folder already created")
    
input_folders = os.listdir(input_base_path)

In [5]:
print(input_folders)

['neg', 'pos']


In [6]:
start = time.time()
print("logging has started")

for folder in input_folders:
    flag_break = False
    folder_path = os.path.join(input_base_path, folder)
    images_list = os.listdir(folder_path)
    
    train_file_path = os.path.join(output_base_path, folder)
    try:
        os.mkdir(train_file_path)
    except Exception as e:
        print("Folder already created")
    print("Processing the images for the behavior: ", folder)
    print("Total images: ", len(images_list))
    
    iteration_ = 0
    mobile_ignored_images = 0
    
    for images in images_list:        
        iteration_ += 1

        img_path = os.path.join(folder_path, images)
        
        img = cv2.imread(img_path)

        img_in = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        results_yolov5 = model_yolov5(img_in, size=640)
        names, scores, converted_boxes, converted_boxes_mobile = processing_yolov5(results_yolov5)
        if len(converted_boxes) == 0:
            print(img_path)
            continue
        
        
        body_features, body_index = bf_ex(img, converted_boxes, converted_boxes_mobile)
        
        # to ignore the using mobile images not having the mobile detected
        if body_features is not None and int(images[4:].split('.')[0]) >= 7154 and int(images[4:].split('.')[0]) <= 8805 and 1.41 - body_features[0][-1] < 0.001:
            mobile_ignored_images += 1
            continue
        
        
        input_features = np.ones((1, num_bf))*(1e-6)
        
        
        if body_features is not None:
#             if len(body_features) >= 2 or len(face_features) >= 2:
#                 continue
            input_features[0, :num_bf] = body_features[0]        
        else:
            #print("Detection Error in: ", img_path)
            continue
        
        #creating the csv of every person
        df = pd.DataFrame(input_features)
        df.to_csv(train_file_path + r'/' + images.split('.')[0] + '.csv', sep=',',index=False, header=False)
        
        if iteration_ != 0 and iteration_ % 40 == 0:
            print("\tImage Processed: ",iteration_)

end = time.time()
print(f"Runtime of the program is {end - start}")

logging has started
Processing the images for the behavior:  neg
Total images:  9241


[ WARN:0@17.347] global /io/opencv/modules/dnn/src/net_impl.cpp (179) setUpNet DNN module was not built with CUDA backend; switching to CPU


../train_latest/neg/neg_4841.png
../train_latest/neg/neg_9046.png
	Image Processed:  40
../train_latest/neg/neg_9025.png
../train_latest/neg/neg_3661.png
	Image Processed:  80
../train_latest/neg/neg_3737.png
	Image Processed:  120
../train_latest/neg/neg_4558.png
../train_latest/neg/neg_9221.png
	Image Processed:  200
../train_latest/neg/neg_4311.png
../train_latest/neg/neg_3588.png
../train_latest/neg/neg_4322.png
	Image Processed:  360
	Image Processed:  400
../train_latest/neg/neg_3881.png
	Image Processed:  440
../train_latest/neg/neg_3699.png
	Image Processed:  480
../train_latest/neg/neg_3483.png
../train_latest/neg/neg_3774.png
	Image Processed:  520
	Image Processed:  560
../train_latest/neg/neg_4118.png
../train_latest/neg/neg_3978.png
	Image Processed:  600
../train_latest/neg/neg_3513.png
../train_latest/neg/neg_4644.png
../train_latest/neg/neg_8998.png
../train_latest/neg/neg_8875.png
	Image Processed:  680
../train_latest/neg/neg_3512.png
	Image Processed:  720
../train_l

	Image Processed:  7360
../train_latest/neg/neg_3825.png
	Image Processed:  7400
../train_latest/neg/neg_8992.png
	Image Processed:  7440
../train_latest/neg/neg_4119.png
../train_latest/neg/neg_8938.png
	Image Processed:  7480
../train_latest/neg/neg_8871.png
	Image Processed:  7520
../train_latest/neg/neg_3514.png
../train_latest/neg/neg_3850.png
	Image Processed:  7560
../train_latest/neg/neg_4419.png
	Image Processed:  7600
../train_latest/neg/neg_4303.png
	Image Processed:  7640
	Image Processed:  7680
../train_latest/neg/neg_8982.png
../train_latest/neg/neg_3775.png
	Image Processed:  7720
../train_latest/neg/neg_2937.png
	Image Processed:  7760
../train_latest/neg/neg_4465.png
	Image Processed:  7800
../train_latest/neg/neg_3539.png
../train_latest/neg/neg_4723.png
	Image Processed:  7840
../train_latest/neg/neg_2640.png
../train_latest/neg/neg_3826.png
	Image Processed:  7920
../train_latest/neg/neg_3192.png
	Image Processed:  8000
	Image Processed:  8040
../train_latest/neg/ne

# Merging the CSVs into one csv

In [7]:
base_folder = output_base_path
output_classes = ['neg', 'pos']
class_index = {'pos' : 1, 'neg': 0}

for class_ in output_classes:
    behavior_index = class_index[class_]
    input_folder = os.path.join(base_folder, class_)
    data = []
    # reading all the csvs inside the folder
    for csvfile in os.listdir(input_folder):
        csvfile_path = os.path.join(input_folder, csvfile)
        df = pd.read_csv(csvfile_path, sep=',', header=None)
        data.extend(df.values.tolist())
    df = pd.DataFrame(data)
    df['class'] = behavior_index
    df.to_csv(base_folder + '/' + class_ + '.csv', sep=',',index=False, header=False)